In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load data

In [2]:
import os
os.system('kaggle datasets download -d uciml/breast-cancer-wisconsin-data -f data.csv');

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## Create X and Y

In [5]:
DIAGNOSIS = 'diagnosis'
TARGET = 'radius_mean'
FEATURES_PREFIX = 'mean'
FEATURES = [feature for feature in df.columns if \
            (feature not in TARGET) and (FEATURES_PREFIX in feature)]
N_FEATURES = len(FEATURES)

In [6]:
y = df[TARGET]
x = df[FEATURES]
x.head()

,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


In [7]:
diagnosis = df[DIAGNOSIS]

## Preprocess data

In [8]:
x /= x.max(axis=0)

In [9]:
x.head()

,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,0.264257,0.651459,0.400240,0.724602,0.803706,0.703140,0.731113,0.795724,0.807779
1,0.452393,0.705040,0.530188,0.518605,0.227678,0.203608,0.348757,0.596053,0.581589
2,0.540988,0.689655,0.481008,0.670747,0.462942,0.462512,0.635686,0.680592,0.615661
3,0.518839,0.411565,0.154378,0.872093,0.821946,0.565604,0.522863,0.854276,1.000000
4,0.365071,0.716711,0.518593,0.613831,0.384482,0.463918,0.518390,0.595066,0.603756


## Split Train, Val & Test

### Train & Val

In [10]:
x_healthy, y_healthy = x[diagnosis == 'B'], y[diagnosis == 'B']

In [11]:
TRAIN_SIZE = int(0.8 * len(x_healthy))

In [12]:
np.random.seed(0)
train_idxs = np.random.choice(x_healthy.index, TRAIN_SIZE, replace=False)
val_idxs = x_healthy.index.drop(train_idxs)

In [13]:
x_train = x_healthy.loc[train_idxs].values
y_train = y_healthy.loc[train_idxs].values
print('# Examples for training:',  len(x_train))

x_val = x_healthy.loc[val_idxs].values
y_val = y_healthy.loc[val_idxs].values
print('# Examples for validation:',  len(x_val))

# Examples for training: 285
# Examples for validation: 72


### Test

In [14]:
x_unhealthy, y_unhealthy = x[diagnosis == 'M'], y[diagnosis == 'M']

In [15]:
x_test = x_unhealthy.values
y_test = y_unhealthy.values
print('# Examples for test:',  len(x_test))

# Examples for test: 212


## Ensemble of models

### Create an ensemble of models

In [16]:
import keras
from keras.layers import *
from keras.models import Model, Sequential
from collections import defaultdict

c:\users\bearch\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
def create_base_model(n_hidden):
    i = Input((N_FEATURES, ))
    h = Dense(n_hidden, kernel_initializer='normal', use_bias=True)(i)
    h = LeakyReLU()(h)
    o = Dense(1)(h)
    return Model(i, o)

In [18]:
models = []
N_MODELS = 5
n_hidden_neurons = 32
for _ in range(N_MODELS):
    models.append(create_base_model(n_hidden_neurons))

In [19]:
models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


### Train ensemble

In [20]:
def train(model, train_data, val_data, lr, batch_size, epochs=50, history=None):
    current_epoch = 0 if history is None else len(history.history['loss'])
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(lr=lr),
        metrics=['mse']
    )
    
    new_history = model.fit(
        train_data[0], train_data[1], epochs=current_epoch+epochs,
        batch_size=batch_size,
        validation_data=val_data,
        initial_epoch=current_epoch,
        verbose=1
    )
    
    if history is not None:
        for key in new_history.history:
            history.history[key].extend(new_history.history[key])
    else:
        history = new_history
    
    return history

In [21]:
histories = defaultdict(lambda: None)

In [ ]:
for lr in [0.5, 0.1, 0.05, 0.01]:
    for i in range(N_MODELS):
        histories[i] = train(models[i], [x_train, y_train], (x_val, y_val), lr, 500, history=histories[i])

Train on 285 samples, validate on 72 samples
Epoch 1/50
285/285 [==============================] - 4s 13ms/step - loss: 148.8573 - mean_squared_error: 148.8573 - val_loss: 18.5348 - val_mean_squared_error: 18.5348
Epoch 2/50
285/285 [==============================] - 0s 17us/step - loss: 20.3959 - mean_squared_error: 20.3959 - val_loss: 949.5780 - val_mean_squared_error: 949.5780
Epoch 3/50
285/285 [==============================] - 0s 19us/step - loss: 889.4991 - mean_squared_error: 889.4991 - val_loss: 38.2756 - val_mean_squared_error: 38.2756
Epoch 4/50
285/285 [==============================] - 0s 17us/step - loss: 34.6568 - mean_squared_error: 34.6568 - val_loss: 64.7996 - val_mean_squared_error: 64.7996
Epoch 5/50
285/285 [==============================] - 0s 17us/step - loss: 66.0322 - mean_squared_error: 66.0322 - val_loss: 110.7904 - val_mean_squared_error: 110.7904
Epoch 6/50
285/285 [==============================] - 0s 20us/step - loss: 110.8513 - mean_squared_error: 110.85

Train on 285 samples, validate on 72 samples
Epoch 1/50
285/285 [==============================] - 0s 1ms/step - loss: 150.8649 - mean_squared_error: 150.8649 - val_loss: 72.4424 - val_mean_squared_error: 72.4424
Epoch 2/50
285/285 [==============================] - 0s 59us/step - loss: 73.5357 - mean_squared_error: 73.5357 - val_loss: 191.8806 - val_mean_squared_error: 191.8806
Epoch 3/50
285/285 [==============================] - 0s 147us/step - loss: 177.4399 - mean_squared_error: 177.4399 - val_loss: 13.4396 - val_mean_squared_error: 13.4396
Epoch 4/50
285/285 [==============================] - 0s 42us/step - loss: 12.3206 - mean_squared_error: 12.3206 - val_loss: 45.4543 - val_mean_squared_error: 45.4543
Epoch 5/50
285/285 [==============================] - 0s 45us/step - loss: 46.8464 - mean_squared_error: 46.8464 - val_loss: 84.0178 - val_mean_squared_error: 84.0178
Epoch 6/50
285/285 [==============================] - 0s 31us/step - loss: 84.4743 - mean_squared_error: 84.4743 -

Train on 285 samples, validate on 72 samples
Epoch 1/50
285/285 [==============================] - 0s 1ms/step - loss: 150.2121 - mean_squared_error: 150.2121 - val_loss: 30.8307 - val_mean_squared_error: 30.8307
Epoch 2/50
285/285 [==============================] - 0s 21us/step - loss: 32.7269 - mean_squared_error: 32.7269 - val_loss: 472.4473 - val_mean_squared_error: 472.4473
Epoch 3/50
285/285 [==============================] - 0s 18us/step - loss: 439.9533 - mean_squared_error: 439.9533 - val_loss: 6.8237 - val_mean_squared_error: 6.8237
Epoch 4/50
285/285 [==============================] - 0s 17us/step - loss: 6.5180 - mean_squared_error: 6.5180 - val_loss: 97.8463 - val_mean_squared_error: 97.8463
Epoch 5/50
285/285 [==============================] - 0s 14us/step - loss: 98.2786 - mean_squared_error: 98.2786 - val_loss: 128.1075 - val_mean_squared_error: 128.1075
Epoch 6/50
285/285 [==============================] - 0s 17us/step - loss: 127.6375 - mean_squared_error: 127.6375 - 

Train on 285 samples, validate on 72 samples
Epoch 1/50
285/285 [==============================] - 0s 2ms/step - loss: 150.0765 - mean_squared_error: 150.0765 - val_loss: 7.9731 - val_mean_squared_error: 7.9731
Epoch 2/50
285/285 [==============================] - 0s 38us/step - loss: 9.5450 - mean_squared_error: 9.5450 - val_loss: 731.9800 - val_mean_squared_error: 731.9800
Epoch 3/50
285/285 [==============================] - 0s 35us/step - loss: 684.9078 - mean_squared_error: 684.9078 - val_loss: 19.8838 - val_mean_squared_error: 19.8838
Epoch 4/50
285/285 [==============================] - 0s 35us/step - loss: 18.0452 - mean_squared_error: 18.0452 - val_loss: 104.5650 - val_mean_squared_error: 104.5650
Epoch 5/50
285/285 [==============================] - 0s 36us/step - loss: 104.7037 - mean_squared_error: 104.7037 - val_loss: 191.3000 - val_mean_squared_error: 191.3000
Epoch 6/50
285/285 [==============================] - 0s 38us/step - loss: 188.7743 - mean_squared_error: 188.774

Train on 285 samples, validate on 72 samples
Epoch 1/50
285/285 [==============================] - 1s 2ms/step - loss: 152.5513 - mean_squared_error: 152.5513 - val_loss: 57.0869 - val_mean_squared_error: 57.0869
Epoch 2/50
285/285 [==============================] - 0s 25us/step - loss: 58.5527 - mean_squared_error: 58.5527 - val_loss: 346.1681 - val_mean_squared_error: 346.1681
Epoch 3/50
285/285 [==============================] - 0s 28us/step - loss: 321.7568 - mean_squared_error: 321.7568 - val_loss: 18.0446 - val_mean_squared_error: 18.0446
Epoch 4/50
285/285 [==============================] - 0s 28us/step - loss: 16.3755 - mean_squared_error: 16.3755 - val_loss: 80.0697 - val_mean_squared_error: 80.0697
Epoch 5/50
285/285 [==============================] - 0s 31us/step - loss: 80.7967 - mean_squared_error: 80.7967 - val_loss: 157.3806 - val_mean_squared_error: 157.3806
Epoch 6/50
285/285 [==============================] - 0s 31us/step - loss: 155.7731 - mean_squared_error: 155.773

...

Train on 285 samples, validate on 72 samples
Epoch 101/150
285/285 [==============================] - 1s 3ms/step - loss: 0.1316 - mean_squared_error: 0.1316 - val_loss: 3.0055 - val_mean_squared_error: 3.0055
Epoch 102/150
285/285 [==============================] - 0s 17us/step - loss: 2.8209 - mean_squared_error: 2.8209 - val_loss: 0.1502 - val_mean_squared_error: 0.1502
Epoch 103/150
285/285 [==============================] - 0s 21us/step - loss: 0.1414 - mean_squared_error: 0.1414 - val_loss: 1.2503 - val_mean_squared_error: 1.2503
Epoch 104/150
285/285 [==============================] - 0s 21us/step - loss: 1.2243 - mean_squared_error: 1.2243 - val_loss: 1.7636 - val_mean_squared_error: 1.7636
Epoch 105/150
285/285 [==============================] - 0s 21us/step - loss: 1.7214 - mean_squared_error: 1.7214 - val_loss: 0.6530 - val_mean_squared_error: 0.6530
Epoch 106/150
285/285 [==============================] - 0s 21us/step - loss: 0.6509 - mean_squared_error: 0.6509 - val_loss: 

Train on 285 samples, validate on 72 samples
Epoch 101/150
285/285 [==============================] - 1s 3ms/step - loss: 0.2440 - mean_squared_error: 0.2440 - val_loss: 4.5147 - val_mean_squared_error: 4.5147
Epoch 102/150
285/285 [==============================] - 0s 28us/step - loss: 4.2213 - mean_squared_error: 4.2213 - val_loss: 0.2560 - val_mean_squared_error: 0.2560
Epoch 103/150
285/285 [==============================] - 0s 28us/step - loss: 0.2560 - mean_squared_error: 0.2560 - val_loss: 1.9177 - val_mean_squared_error: 1.9177
Epoch 104/150
285/285 [==============================] - 0s 28us/step - loss: 1.9203 - mean_squared_error: 1.9203 - val_loss: 2.7686 - val_mean_squared_error: 2.7686
Epoch 105/150
285/285 [==============================] - 0s 31us/step - loss: 2.7453 - mean_squared_error: 2.7453 - val_loss: 1.1241 - val_mean_squared_error: 1.1241
Epoch 106/150
285/285 [==============================] - 0s 29us/step - loss: 1.1474 - mean_squared_error: 1.1474 - val_loss: 

Train on 285 samples, validate on 72 samples
Epoch 101/150
285/285 [==============================] - 1s 3ms/step - loss: 0.1599 - mean_squared_error: 0.1599 - val_loss: 5.2957 - val_mean_squared_error: 5.2957
Epoch 102/150
285/285 [==============================] - 0s 28us/step - loss: 5.0867 - mean_squared_error: 5.0867 - val_loss: 0.1948 - val_mean_squared_error: 0.1948
Epoch 103/150
285/285 [==============================] - 0s 31us/step - loss: 0.1972 - mean_squared_error: 0.1972 - val_loss: 2.4708 - val_mean_squared_error: 2.4708
Epoch 104/150
285/285 [==============================] - 0s 26us/step - loss: 2.3387 - mean_squared_error: 2.3387 - val_loss: 3.3373 - val_mean_squared_error: 3.3373
Epoch 105/150
285/285 [==============================] - 0s 28us/step - loss: 3.1596 - mean_squared_error: 3.1596 - val_loss: 1.1021 - val_mean_squared_error: 1.1021
Epoch 106/150
285/285 [==============================] - 0s 31us/step - loss: 1.0412 - mean_squared_error: 1.0412 - val_loss: 

Train on 285 samples, validate on 72 samples
Epoch 151/200
285/285 [==============================] - 1s 3ms/step - loss: 0.0837 - mean_squared_error: 0.0837 - val_loss: 0.1692 - val_mean_squared_error: 0.1692
Epoch 152/200
285/285 [==============================] - 0s 21us/step - loss: 0.1715 - mean_squared_error: 0.1715 - val_loss: 0.0681 - val_mean_squared_error: 0.0681
Epoch 153/200
285/285 [==============================] - 0s 17us/step - loss: 0.0722 - mean_squared_error: 0.0722 - val_loss: 0.0897 - val_mean_squared_error: 0.0897
Epoch 154/200
285/285 [==============================] - 0s 17us/step - loss: 0.0880 - mean_squared_error: 0.0880 - val_loss: 0.1236 - val_mean_squared_error: 0.1236
Epoch 155/200
285/285 [==============================] - 0s 25us/step - loss: 0.1188 - mean_squared_error: 0.1188 - val_loss: 0.0744 - val_mean_squared_error: 0.0744
Epoch 156/200
285/285 [==============================] - 0s 18us/step - loss: 0.0739 - mean_squared_error: 0.0739 - val_loss: 

Train on 285 samples, validate on 72 samples
Epoch 151/200
285/285 [==============================] - 1s 3ms/step - loss: 0.0552 - mean_squared_error: 0.0552 - val_loss: 0.1436 - val_mean_squared_error: 0.1436
Epoch 152/200
285/285 [==============================] - 0s 24us/step - loss: 0.1408 - mean_squared_error: 0.1408 - val_loss: 0.0536 - val_mean_squared_error: 0.0536
Epoch 153/200
285/285 [==============================] - 0s 21us/step - loss: 0.0535 - mean_squared_error: 0.0535 - val_loss: 0.0910 - val_mean_squared_error: 0.0910
Epoch 154/200
285/285 [==============================] - 0s 17us/step - loss: 0.0873 - mean_squared_error: 0.0873 - val_loss: 0.1071 - val_mean_squared_error: 0.1071
Epoch 155/200
285/285 [==============================] - 0s 18us/step - loss: 0.1023 - mean_squared_error: 0.1023 - val_loss: 0.0659 - val_mean_squared_error: 0.0659
Epoch 156/200
285/285 [==============================] - 0s 17us/step - loss: 0.0639 - mean_squared_error: 0.0639 - val_loss: 

## Predict target feature using ensemble

In [ ]:
y_train_preds = []
y_val_preds = []
y_test_preds = []

for model in models:
    y_train_preds.append(model.predict(x_train))
    y_val_preds.append(model.predict(x_val))
    y_test_preds.append(model.predict(x_test))

## Use ensemble variance to predict diagnosis 

In [ ]:
y_train_preds_var = np.var(y_train_preds, axis=0)
y_val_preds_var = np.var(y_val_preds, axis=0)
y_test_preds_var = np.var(y_test_preds, axis=0)

In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(y_train_preds_var, color='b',  bins=50, label='train')
plt.hist(y_val_preds_var, fc=(1, 0, 0, 0.5), bins=50, label='val')
plt.hist(y_test_preds_var, fc=(0, 1, 0, 0.5), bins=500, label='test')
plt.legend()
plt.show()

### Select variance threshold

In [ ]:
threshold_var = 1.5 * y_train_preds_var.mean()

In [ ]:
train_acc = (y_train_preds_var < threshold_var).mean()
print(train_acc)

### Validation & Test

In [ ]:
val_acc = (y_val_preds_var < threshold_var).mean()
print(val_acc)

In [ ]:
test_acc = (y_test_preds_var >= threshold_var).mean()
print(test_acc)

Done!